In [ ]:
# Import some python libraries
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import intake

In [ ]:
# Setup a dask cluster
from dask.distributed import Client
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
client = Client(cluster)
client

In [ ]:
col = intake.open_esm_datastore("https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json")

In [ ]:
variable_id = 'tas'
table_id='Amon'
source_id = 'CESM1-1-CAM5-CMIP5'

# Get hindcast experiment

In [ ]:
cat_cmip = col.search(experiment_id=['dcppA-hindcast'],
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id,
                     )
cat_cmip.df['dcpp_init_year'] = cat_cmip.df.dcpp_init_year.astype(int)
cat_cmip

In [ ]:
members = sorted(list(cat_cmip.df.member_id.unique()))[:20]
inits = list(np.arange(1970,1990))

In [ ]:
cat_cmip = col.search(experiment_id=['dcppA-hindcast'],
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id,
                 member_id = members,
                 dcpp_init_year=inits
                     )
cat_cmip.df['dcpp_init_year'] = cat_cmip.df.dcpp_init_year.astype(int)

In [ ]:
def pre(ds,var='tas'):
    ds['time']=np.arange(1,1+ds.time.size)
    return ds[var].squeeze().to_dataset(name=var)

In [ ]:
dset_dict = cat_cmip.to_dataset_dict(zarr_kwargs={'consolidated': True},preprocess=pre,cdf_kwargs={'decode_times':False})
list(dset_dict.keys())

In [ ]:
hind = dset_dict['DCPP.NCAR.CESM1-1-CAM5-CMIP5.dcppA-hindcast.Amon.gn']

In [ ]:
hind = hind.rename({'member_id':'member','dcpp_init_year':'init','time':'lead'})

In [ ]:
# throw away first two leads and create annual means
hind=hind.isel(lead=slice(2,None))
hind['lead']=xr.cftime_range(start='2000',freq='MS',periods=hind.lead.size)
hind_ym = hind.groupby('lead.year').mean().rename({'year':'lead'})
hind_ym['lead']=np.arange(1,1+hind_ym.lead.size)

In [ ]:
hind_ym = hind_ym.chunk({'init':-1,'member':-1,'lead':1}).persist()

hind_ym = hind_ym['tas']

## Get historical

In [ ]:
cat_cmip_hist = col.search(experiment_id=['historical'],  # pick the `historical` forcing experiment
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id,
                 )


## Get assimilation

In [ ]:
cat_cmip_hist = col.search(experiment_id=['dcppA-assim'],  # pick the `historical` forcing experiment
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id
                          )

## Get observations

## Skill

In [ ]:
from climpred.prediction import compute_hindcast

## Significant skill

In [ ]:
from climpred.bootstrap import bootstrap_hindcast